# Principal Componenet Analysis (PCA)
The following code uses Numpy's `svd()` function to obtain all the principal components of the training set, then extract the two unit vectors that define the first two PCs.
**Computational Complexity** = $O(m \times n^2) + O(n^3)$

In [1]:
import numpy as np

In [ ]:
X_centered = X - X.mean(axis=0)
U, s, VT = np.linalg.svd(X_centered)
c1 = VT.T[:, 0]
c2 = Vt.T[:, 1]

### To project the training set onto the hyperplane
$$X_{d\_proj} = XW_d$$
where:
- $X_{d\_proj}$ reduced dataset of dimensionality $d$
- $X$ training set matrix
- $W_d$ matrix containing the first $d$ columns of $V$

In [ ]:
Wd = VT.T[:, :2]
X_2d = X_centered.dot(Wd)  # X2d i.e. X is now reduced to 2 dimensional

## Using Scikit-learn

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_componenents=2)
X_2d = pca.fit_transform(X)

After fitting `PCA` transformer; `components_` attribute holds the **transpose** of $W_d$.

In [ ]:
#The first principle componenet
pca.componenets.T[:, 0]

### explained variance ratio
`PCA().explained_variance_ratio_`
#### Indicates the proportion to the dataset's variance that lies along each principal component.

In [ ]:
pca.explined_variance_ratio

### Choosing the Right Number of Dimensions
#### Choose the number of dimensions that add up to suffieciently large portion of the variance.

In [ ]:
pca = PCA()
pca.fit(X)
cumsum = np.cumsum(pca.explaines_variance_ratio)
req_dimensions = np.argmax(cumsum) + 1          # index start at '0'

#### Yet BETTER way to do the this
Instead of setting `n_compnents`= req_dimension`,

Put the `n_components` a float no between 0.0 and 1.0, for the amount of variance you want to preserve(typically 95% for next process OR 2-3 for DataViz)

In [ ]:
pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X_train)

#### Another option is to plot explained variance ratio as a function of the number of dimensions; plot `cumsum`
There ususally be an elbow  in the curve, where the explained variance stops growing fast.

## PCA for Compression
To recover from data from compression: `PCA().inverse_transform` 

$$X_{recovered} = X_{d\_proj}{W_d}^T$$
The mean squared distance between the original data and the reconstructed data is called **reconstruction error**


In [ ]:
pca = PCA(n_componenets=154)
X_reduced = pca.fit_transform(X_train)
X_recovered = pca.inverse_transform(X_reduced)

## Randomized PCA
`svd_solver = 'randomized'`

Has a reduced complexity of the first *d* principle components from full SVD approach of $O(m \times n^2) + O(n^3)$.

**Computational Complexity** = $O(m \times d^2) + O(d^2)$

- When $d$ is much smaller than $m$.

By default `svd_solver = 'auto'`: Automatically use randomized PCA **IF**
- $m$ or $n$ is greater than 500 **AND** $d$ is less than 80% of $m$ or $n$. 
- or else use full SVD approach

Set `svd_solver = 'full'` to force use of full SVD.

In [ ]:
rnd_pca = PCA(n_components=154, svd_solver='randomized')
X_reduced = rnd_pca.fit_tranform(X_train)

## Incremental PCA (IPCA)
`IncrementalPCA`

Allows training set to be split into mini-batches and feed an IPCA algorithm one mini-batch at a time.
- Use `partial_fit` rather than `fit`.

### Using `np.arrary_split()`.

In [ ]:
from sklearn.decomposition import IncrementalPCA

n_batches =  100
inc_pca = IncrementalPCA(n_components=154)

for X_batch in np.array_split(X_train, n_batches):
    inc_pca.partial_fit(X_batch)
    
n_reduced = inc_pca.transform(X_train)

### Using `np.memmap()`
**The class only loads the data it needs in memory, when it needs it.** Which allows you to manipulate a large array stored in a binary file on disk as if it were entirely in memory.

This maked it possible t call the usual `fit()` method.

In [ ]:
X_mm = np.memmap(filename, dtype='float32', mode='readonly', shape=(m, n))

batch_size = m // n_batches
inc_pca = IncrementalPCA(n_componenets=154, batch_ssize=batch_size)
inc_pca.fit(X_mm)

# Kernel PCA
An unsupervised learning algorithm

In [ ]:
from sklearn.decomposition import KernelPCA

rbf_pca = kernelPCA(n_components=2, kernel='rbf', gamma=0.04)
X_reduced = rbf_pca.fit_transform(X)

## Selecting a Kernel and Tuning Hyperparameters
Since kPCA is an unsupervised algorithm, there is no obvious performance to help you select the best kernel and hyperparameter value. But since dimensionality reduction is often preparation step, after which the reduced dataset is send to supervised algorithm, 
#### we can measure ML model performance and selct the best hyperparameter values for the kPCA.

The following code creates a two step pipeline, first reducing dimensionality to two dimensions using kPCA, then applying Logistic Regression for classification. Then it uses GridSearchCV to find the best kernel and gamma values for kPCA in order to get the best calssification accuracy at the end of the pipeline.

In [ ]:
from sklearn.decomposition import KernelPCA
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

clf = Pipeline([
    ('kpca', KernelPCA(n_components=2)),
    ('log_reg', LogisticRegression())
])
param_grid = [{
    'kpca_gamma': np.linspace(0.83, 0.05, 10),
    'kpca_kernel': ['rbf', 'sigmoid']
}]

grid_search = GridSearchCV(clf, param_grid, cv=3)
grid_search.fit(X, y)

print(grid_search.best_params_)

#### Another approach is to select the kernel and hyperparameters that yield the lowest reconstruction error.

In [ ]:
rbf_pca = KernelPCA(n_components=2, kernel='rbf', gamma=0.0433,
                    fit_inverse=True)
X_reduced = rbf_pca.fit_transform(X)
X_preimage = rbf_pca.inverse_transform(X_reduced)

Then compute the recontruction pre-image error

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(X, X_preimage)

Now you can use grid search with cross calidation to find the kernel and hyperparameters that minimize this error.

# Locally Linear Embedding (LLE)
- another powerful **Nonlinear dimensionality reduction** (NLDR) technique.
- <mark>Manifold Learning technique</mark> that does not rely on projections, like the previous algorithm do.
- Particularly good at unrolling twisted manifolds.

In a nutshell, LLE works by first measuring how each training instance linearly relates to its closest neighbors (c.n.), and then looking for a low-dimensional representation of the training set where these local relationships are best preserved.

In [ ]:
from sklearn.manifold import LocallyLinearEmbedding

lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10)
X_reduced = lle.fit_transform(X)

## How LLE works
### Step one: Linearly modeling local relationships
- For each training instance $x^{(i)}$, the algorithm identifies its $k$ closest neighbors (here $k$=10).
- Then it tries to reconstruct $x^{(i)}$ as linear function of these neighbors.
  
  It finds the weights $w_{i,j}$ such that the squared distance between $x^{(i)}$ and $\sum^{m}_{j=1}w_{i,j} x^{(j)}$ is as small as possible, assuming $w_{i,j}=0$ if $x^{(i)}$ is not one of the $k$ closest neighbors.
  
### Step two: Reducing dimensionality while preserving relationships
Map the training instances into a $d$-dimensional space (where $d$ < $n$) while preserving these local relationships.
- If $z^{(i)}$ is the image of $x^{(i)}$ in this $d$-dimensional space, then we want the squared distance between $z^{(i)}$ and $\sum^{m}_{j=1}\hat{w}_{i, j}z^{(i)}$ to be as small as possible.
- Keeping the weights fixed and finding the optimal position of the instance's image in the low dimensional space.

#### Computational Complexity     
- For finding the k nearest neighbors = $O(mlog(m)nlog(k)$
- For optimizing the weights = $O(mnk^3)$
- For constructing the low dimensional representations = $O(dm^2)$

The $m^2$ in the last term makes this algorithm scale poorly to very large datasets.